**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [128]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json


from keras.utils import to_categorical
from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten, Input

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

Epsilon is essential for exploration (exploitation - exploration tradeoff).

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale = 16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size * self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3)) + 128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:] = 256
        b[-2:,:,:] = 0
        b[:,-2:,:] = 0
        b[:2,:,:] = 0
        b[:,:2,:] = 0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:] = b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x + 1
            else:
                self.x = self.x - 1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x - 2:self.x + 3,self.y - 2:self.y + 3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5 * np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0 * np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time + 2, self.grid_size * self.scale, self.grid_size * self.scale, 3))


        malus[bonus > 0] = 0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:] = -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=30 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

Position array illustrates where the player is on the grid: 0 for its position, -1 for all the others.

Board array describes where the bonus and malus are (0.5 for bonus squares = cheese, -1 for malus squares = poison). The Board array is also used to construct the state visible to the agents: the square of 5x5 with the agent’s position in the middle.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.choice(list(range(0, self.n_action)))

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0
        
        while not game_over:
            # The agent performs an action
            action = agent.learned_act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward
        
        # Save as a mp4
        env.draw(prefix + str(e))

        # Update stats
        score = score + win - lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score / (1 + e)))
    print('Final score: ' + str(score / epochs))

In [11]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
#HTML(display_videos('random0.mp4'))

Win/lose count 8.0/13.0. Average score (-5.0)
Win/lose count 11.0/19.0. Average score (-6.5)
Win/lose count 14.0/17.0. Average score (-5.333333333333333)
Win/lose count 14.0/11.0. Average score (-3.25)
Win/lose count 7.5/21.0. Average score (-5.3)
Win/lose count 12.5/13.0. Average score (-4.5)
Win/lose count 7.5/7.0. Average score (-3.7857142857142856)
Win/lose count 13.0/20.0. Average score (-4.1875)
Win/lose count 10.0/14.0. Average score (-4.166666666666667)
Win/lose count 10.0/11.0. Average score (-3.85)
Final score: -3.85


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




1) Proof that 

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

\begin{align*}
    Q^\pi(s,a) &= E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> = E_{p^{\pi}}[\sum_{t = 0}^{T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] = E_{p^{\pi}}[r(s, a) + \sum_{t = 1}^{T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] = \newline
    &= E_{(s',a')\sim p(.|s,a)}[E_{p^{\pi}}[r(s, a) + \sum_{t = 1}^{T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]] =  E_{(s',a')\sim p(.|s,a)}[r(s, a) + \gamma E_{p^{\pi}}[ \sum_{t = 0}^{T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a']] = \newline
    &= E_{(s',a')\sim p(.|s,a)} [r(s, a) + \gamma Q^\pi(s',a')]
\end{align*}


2) We assume that the optimal policy $\pi^*$ exists. We then write the Bellman equation (derived above) for $\pi^{*}$, and $Q^{*} = Q^{\pi^*}$.

\begin{align} Q^{*}(s, a) \ &= Q^{\pi^*}(s, a) \\ &= E_{(s',a')\sim \pi*(.|s,a)}[r(s,a)+\gamma Q^{\pi^*}(s',a')] \\ &= \max_{\pi} E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] \\ &= r(s,a)+ \gamma \max_{\pi} \sum_{s', a'}{\pi(a_{t+1} = a' | s_{t+1} = s') p(s_{t+1}=s'|s_t=s, a_t=a) Q^{\pi}(s',a')} \\ &= r(s,a)+ \gamma \max_{\pi} \sum_{s'}{p(s_{t+1}=s'|s_t=s, a_t=a) Q^{\pi}(s',a')} \end{align}

We now prove that we can exchange the sum and the maximum operations, by proving the following inequalities:

\begin{align}
\max_{\pi} \sum_{s'}{p(s_{t+1}=s'|s_t=s, a_t=a) Q^{\pi}(s',a')} \leq \sum_{s'}{p(s_{t+1}=s'|s_t=s, a_t=a) \max_{\pi} Q^{\pi}(s',a')} 
\end{align}

\begin{align}
\sum_{s'}{p(s_{t+1}=s'|s_t=s, a_t=a) \max_{\pi} Q^{\pi}(s',a')} \leq \max_{\pi} \sum_{s'}{p(s_{t+1}=s'|s_t=s, a_t=a) Q^{\pi}(s',a')}
\end{align}


The first inequality is trivial (max of sums lower than sum of max).

The second can be proven by choosing ${\pi}^{'}(s',a')$ such that $\max_{\pi}{Q^{\pi}(s', a')} = {Q^{\pi^{'}}(s', a')}$; then:

\begin{align}
\sum_{s'}{p(s_{t+1}=s'|s_t=s, a_t=a) \max_{\pi} Q^{\pi}(s',a')}\ &= \sum_{s'}{p(s_{t+1}=s'|s_t=s, a_t=a) Q^{\pi^{'}}(s',a')} \\ & \leq \max_{\pi} \sum_{s'}{p(s_{t+1}=s'|s_t=s, a_t=a) Q^{\pi}(s',a')}
\end{align}

So we can exchange the sum and the maximum operations: 

\begin{align}
\sum_{s'}{p(s_{t+1}=s'|s_t=s, a_t=a) \max_{\pi} Q^{\pi}(s',a')} = \max_{\pi} \sum_{s'}{p(s_{t+1}=s'|s_t=s, a_t=a) Q^{\pi}(s',a')}
\end{align}

We then obtain, after exchanging the sum and the maximum operations: 

\begin{align} Q^{*}(s,a) &= r(s,a) + \gamma \sum{p(s_{t+1}=s' |s_t = s, a_t = a )\max_{\pi, a'} Q^{\pi}(s',a')} \\ &= r(s,a) + \gamma \sum{p(s_{t+1}=s' |s_t = s, a_t = a )\max_{a'} \max_{\pi} Q^{\pi}(s',a')} \\ &=  r(s,a) + \gamma \sum{p(s_{t+1}=s' |s_t = s, a_t = a )\max_{a'}Q^{*}(s',a')} \\ &= r(s,a) + \sum{p(s_{t+1}=s' |s_t = s, a_t = a ) \gamma \max_{a'}Q^{*}(s',a')} \\ &= r(s,a) + E_{s'\sim \pi^*(.|s,a)}[\gamma \max_{a'}Q^{*}(s',a')] \\ &= E_{s'\sim \pi^*(.|s,a)}[r(s,a) + \gamma \max_{a'}Q^{*}(s',a')] \end{align}


3) Deduce that a plausible objective is:

\begin{equation*}
    \mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

$\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}$ is a plausible objective, because it takes non-negative values and it is easy to see that for the optimal policy $Q = Q^{*}$ it becomes 0 (using the proof above). It is also easy to see that if $Q(s, a)$ is very close in value to $Q^{*}(s, a)$ for every $(s, a)$, then the objective's value will approach 0.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [12]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()
        #index to refill memory circularly (first in, first out) after memory full
        self.index = 0

    def remember(self, m):
        if len(self.memory) < self.max_memory:
            self.memory.append(m)
        else:
            self.memory[self.index] = m
            self.index = (self.index + 1) % self.max_memory

    def random_access(self):
        if len(self.memory) > 0:    
            return self.memory[np.random.choice(range(len(self.memory)))]
        else:
            return None

***
The pipeline we will use for training is given below:

In [13]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        total_frames = 0
        
        losses = []

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
                
            total_frames += 1

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
            
            losses.append(loss)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix + str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose {}/{} ({}), total frames {}"
              .format(e, epoch, np.mean(losses), win, lose, win-lose, total_frames))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [14]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size
        
        self.epsilon = epsilon

    def learned_act(self, s):

        return np.ndarray.argmax(self.model.predict(np.expand_dims(s, axis=0)), axis=1)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            s, n_s, a, r, game_over = self.memory.random_access()

            input_states[i] = s
            
            if game_over:
                ######## FILL IN
                target_q[i, :] = self.model.predict(np.expand_dims(s, axis=0))#[0]
                target_q[i, a] = r
            else:
                ######## FILL IN
                target_q[i, :] = self.model.predict(np.expand_dims(s, axis=0))#[0]
                target_q[i, a] = r + self.discount * np.max(self.model.predict(np.expand_dims(n_s, axis=0)))
            
            
            
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        model = Sequential()
        model.add(Reshape((5*5*self.n_state, ), input_shape=(5,5,self.n_state)))
        model.add(Dense(16, input_dim=5*5*self.n_state, activation='relu'))
        model.add(Dense(12, activation='relu'))
        model.add(Dense(4, activation=None))
        
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        
        self.model = model
        

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))







Epoch 000/030 | Loss 0.0385 | Win/lose 4.0/7.0 (-3.0), total frames 201
Epoch 001/030 | Loss 0.0295 | Win/lose 6.0/4.0 (2.0), total frames 201
Epoch 002/030 | Loss 0.0246 | Win/lose 3.0/6.0 (-3.0), total frames 201
Epoch 003/030 | Loss 0.0190 | Win/lose 0.5/2.0 (-1.5), total frames 201
Epoch 004/030 | Loss 0.0156 | Win/lose 5.0/2.0 (3.0), total frames 201
Epoch 005/030 | Loss 0.0147 | Win/lose 3.0/6.0 (-3.0), total frames 201
Epoch 006/030 | Loss 0.0176 | Win/lose 3.5/4.0 (-0.5), total frames 201
Epoch 007/030 | Loss 0.0134 | Win/lose 4.5/3.0 (1.5), total frames 201
Epoch 008/030 | Loss 0.0137 | Win/lose 1.5/5.0 (-3.5), total frames 201
Epoch 009/030 | Loss 0.0119 | Win/lose 4.0/5.0 (-1.0), total frames 201
Epoch 010/030 | Loss 0.0135 | Win/lose 1.0/1.0 (0.0), total frames 201
Epoch 011/030 | Loss 0.0133 | Win/lose 6.5/4.0 (2.5), total frames 201
Epoch 012/030 | Loss 0.0134 | Win/lose 3.0/3.0 (0.0), total frames 201
Epoch 013/030 | Loss 0.0116 | Win/lose 5.5/3.0 (2.5), total fram

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [15]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(16, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
        model.add(Conv2D(16, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
        model.add(Flatten())
        model.add(Dense(4, activation=None))
                
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        
        self.model = model

In [16]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))







Epoch 000/030 | Loss 0.0079 | Win/lose 2.0/3.0 (-1.0), total frames 201
Epoch 001/030 | Loss 0.0048 | Win/lose 2.5/5.0 (-2.5), total frames 201
Epoch 002/030 | Loss 0.0066 | Win/lose 2.0/8.0 (-6.0), total frames 201
Epoch 003/030 | Loss 0.0062 | Win/lose 4.5/5.0 (-0.5), total frames 201
Epoch 004/030 | Loss 0.0068 | Win/lose 4.0/3.0 (1.0), total frames 201
Epoch 005/030 | Loss 0.0062 | Win/lose 3.0/3.0 (0.0), total frames 201
Epoch 006/030 | Loss 0.0065 | Win/lose 3.0/2.0 (1.0), total frames 201
Epoch 007/030 | Loss 0.0057 | Win/lose 5.5/3.0 (2.5), total frames 201
Epoch 008/030 | Loss 0.0053 | Win/lose 2.0/0 (2.0), total frames 201
Epoch 009/030 | Loss 0.0050 | Win/lose 3.5/4.0 (-0.5), total frames 201
Epoch 010/030 | Loss 0.0058 | Win/lose 2.0/3.0 (-1.0), total frames 201
Epoch 011/030 | Loss 0.0060 | Win/lose 2.0/1.0 (1.0), total frames 201
Epoch 012/030 | Loss 0.0070 | Win/lose 10.0/4.0 (6.0), total frames 201
Epoch 013/030 | Loss 0.0093 | Win/lose 3.5/0 (3.5), total frames 2

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [17]:
temp_range = list(np.arange(0,9) * 0.1 + 0.1)

for temperature in temp_range:
    
    print()
    print("Temperature = ", temperature)
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    env = Environment(grid_size=size, max_time=T,temperature=temperature)
    agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
    agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel_' + str(temperature) + '.json')

    agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
    agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel_' + str(temperature) + '.json')
    print('Test of the CNN')
    test(agent_cnn,env,epochs_test,prefix='cnn_test')
    print('Test of the FC')
    test(agent_fc,env,epochs_test,prefix='fc_test')





Temperature =  0.1
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Test of the CNN
Win/lose count 1.5/0. Average score (1.5)
Win/lose count 1.0/0. Average score (1.25)
Win/lose count 0.5/0. Average score (1.0)
Win/lose count 1.0/0. Average score (1.0)
Win/lose count 0.5/0. Average score (0.9)
Win/lose count 1.0/0. Average score (0.9166666666666666)
Win/lose count 2.5/0. Average score (1.1428571428571428)
Win/lose count 0.5/0. Average score (1.0625)
Win/lose count 0.5/0. Average score (1.0)
Win/lose count 1.0/0. Average score (1.0)
Final score: 1.0
Test of the FC
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 1.0/1.0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0.5/1.0. Average score (-0.08333333333333333)
Win/lose count 0.5/0. Average score (0.0)
Win/lose count 1.0/0. Average score (0.125)
Win/lose count 0.5/0. Average score (0.16666666666666666)
Win/lose count 0/0. Average score (0.15

Win/lose count 7.0/2.0. Average score (3.375)
Win/lose count 5.0/3.0. Average score (3.1)
Win/lose count 6.0/4.0. Average score (2.9166666666666665)
Win/lose count 3.0/0. Average score (2.9285714285714284)
Win/lose count 1.5/1.0. Average score (2.625)
Win/lose count 0.5/0. Average score (2.388888888888889)
Win/lose count 3.5/0. Average score (2.5)
Final score: 2.5

Temperature =  0.9
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Test of the CNN
Win/lose count 15.5/0. Average score (15.5)
Win/lose count 8.5/0. Average score (12.0)
Win/lose count 15.0/0. Average score (13.0)
Win/lose count 8.0/0. Average score (11.75)
Win/lose count 2.5/0. Average score (9.9)
Win/lose count 26.5/0. Average score (12.666666666666666)
Win/lose count 11.0/0. Average score (12.428571428571429)
Win/lose count 1.5/0. Average score (11.0625)
Win/lose count 7.5/1.0. Average score (10.555555555555555)
Win/lose count 16.0/0. Average score (11.1)
Final score: 11.1
Test of the FC
Win/lose count 1.5/0. Average score (1.5)
Win/lose c

In [20]:
#HTML(display_videos('cnn_test10.mp4'))

In [19]:
#HTML(display_videos('fc_test10.mp4'))

The results achieved with the CNN model and the Fully Convolutional network (the results are averaged over 10 epochs), with respect to different values of the temperature are illustrated in the following table.



| Temperature   |     Model     | Final score (averaged over 10 epochs) | 
| ------------- | ------------- |-------------------------------------
| 0.1           |      CNN      |             1.0
| 0.1           |      FC       |             0.15    
| 0.2           |      CNN      |             1.3
| 0.2           |      FC       |            -0.75
| 0.3           |      CNN      |             2.25
| 0.3           |      FC       |             0.15
| 0.4           |      CNN      |             1.95
| 0.4           |      FC       |             0.5
| 0.5           |      CNN      |             2.8
| 0.5           |      FC       |             0.4
| 0.6           |      CNN      |             3.15
| 0.6           |      FC       |             0.55
| 0.7           |      CNN      |             4.45
| 0.7           |      FC       |             0.2
| 0.8           |      CNN      |             5.2
| 0.8           |      FC       |             2.5
| 0.9           |      CNN      |             11.1
| 0.9           |      FC       |             2.9



As we can observe from the table, the CNN outperforms the FC model. We can also observe that the final score improves as the temperature increases. This might be explained by the fact that without having a policy which explores sufficiently, the training is noticeably difficult when having a low temperature, as the agent receives more rarely a positive or a negative reward. Also, we can observe from the behaviour of the agent that in both cases (when using both models), the agent frequently gets stuck into a region and it oscillates between two states, without exploring other possible states. This issue is due to the lack of a good exploration strategy (actually, this issue is more evidenced when the agent does not observe a reward around him and, obviously, this happens more often when the temperature is lower, because the rewards are more sparse in that case).

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [21]:

def train_explore(agent,env,epoch,prefix=''):
    score = 0
    loss = 0
    
    epsilon_decay=0.99

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        total_frames = 0
        
        losses = []

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)
            

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
                
            total_frames += 1

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
            
            agent.set_epsilon(agent.epsilon * epsilon_decay)
            
            losses.append(loss)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose {}/{} ({}), total frames {}"
              .format(e, epoch, np.mean(losses), win, lose, win-lose, total_frames))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
    
    

class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        super().__init__(grid_size, max_time, temperature)
        self.malus_position = np.zeros((self.grid_size,self.grid_size))


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5 * np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0 * np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time + 2, self.grid_size * self.scale, self.grid_size * self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:] = -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.malus_position[self.x, self.y] = 0.1
    
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state    

In [22]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
#HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/030 | Loss 0.0203 | Win/lose 7.5/28.200000000000063 (-20.700000000000063), total frames 201
Epoch 001/030 | Loss 0.0102 | Win/lose 2.2/19.90000000000001 (-17.70000000000001), total frames 201
Epoch 002/030 | Loss 0.0076 | Win/lose 3.0/21.60000000000003 (-18.60000000000003), total frames 201
Epoch 003/030 | Loss 0.0059 | Win/lose 4.199999999999999/17.899999999999984 (-13.699999999999985), total frames 201
Epoch 004/030 | Loss 0.0054 | Win/lose 2.3/20.600000000000023 (-18.300000000000022), total frames 201
Epoch 005/030 | Loss 0.0058 | Win/lose 2.5/20.20000000000002 (-17.70000000000002), total frames 201
Epoch 006/030 | Loss 0.0063 | Win/lose 7.8000000000000025/18.799999999999997 (-10.999999999999995), total frames 201
Epoch 007/030 | Loss 0.0053 | Win/lose 6.500000000000001/19.400000000000006 (-12.900000000000006), total frames 201
Epoch 008/030 | Loss 0.0049 | Win/lose 5.2/18.599999999999994 (-13.399999999999995), total frames 201
Epoch 009/030 | Loss 0.0051 | Win/lose 1.2000

In [138]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
#HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 12.0/0. Average score (12.0)
Win/lose count 7.5/0. Average score (9.75)
Win/lose count 1.0/0. Average score (6.833333333333333)
Win/lose count 6.5/0. Average score (6.75)
Win/lose count 10.5/0. Average score (7.5)
Win/lose count 6.5/0. Average score (7.333333333333333)
Win/lose count 11.0/0. Average score (7.857142857142857)
Win/lose count 1.5/0. Average score (7.0625)
Win/lose count 3.0/0. Average score (6.611111111111111)
Win/lose count 11.0/0. Average score (7.05)
Final score: 7.05


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

In [152]:
new_memory = Memory(10000)
n_episodes = 30

# Number of won games
score = 0
    
for e in range(n_episodes):

    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
        
    while not game_over:
        # The agent performs an action
        action = agent.learned_act(state)

        new_memory.remember((state, action))

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

    # Update stats
    score = score + win-lose

    print("Win/lose count {}/{}. Average score ({})"
          .format(win, lose, score/(1+e)))
print('Final score: '+str(score/n_episodes))

Win/lose count 8.5/2.0. Average score (6.5)
Win/lose count 0.5/0. Average score (3.5)
Win/lose count 5.0/0. Average score (4.0)
Win/lose count 4.0/0. Average score (4.0)
Win/lose count 3.5/0. Average score (3.9)
Win/lose count 4.5/0. Average score (4.0)
Win/lose count 9.0/0. Average score (4.714285714285714)
Win/lose count 5.5/0. Average score (4.8125)
Win/lose count 7.0/0. Average score (5.055555555555555)
Win/lose count 4.0/0. Average score (4.95)
Win/lose count 1.5/0. Average score (4.636363636363637)
Win/lose count 3.5/0. Average score (4.541666666666667)
Win/lose count 11.5/0. Average score (5.076923076923077)
Win/lose count 3.0/0. Average score (4.928571428571429)
Win/lose count 8.0/0. Average score (5.133333333333334)
Win/lose count 3.0/0. Average score (5.0)
Win/lose count 8.5/0. Average score (5.205882352941177)
Win/lose count 9.5/0. Average score (5.444444444444445)
Win/lose count 3.5/0. Average score (5.342105263157895)
Win/lose count 3.5/0. Average score (5.25)
Win/lose cou

In [164]:
#Supervised learning: input(state) -> label (action)

class ImitatingAgent(Agent):
    def __init__(self, memory,*args, lr=0.1, **kwargs):
        super(Agent, self).__init__(*args,**kwargs)       


        self.mem = memory

        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(16, kernel_size=(3, 3), strides=(1, 1), activation='relu', input_shape=(5,5,3)))
        model.add(BatchNormalization())
        model.add(Conv2D(16, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
        model.add(Flatten())
        model.add(Dense(4, activation="softmax"))
                
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), keras.losses.categorical_crossentropy, metrics = ["accuracy"])

        self.model = model
        
        print(self.model.summary())


    def imitate(self):
        X = []
        Y = []
        for i in range(len(self.mem.memory)):
            state = self.mem.memory[i][0]
            action = self.mem.memory[i][1]
            X.append(state)
            Y.append(action)

        X = np.array(X)
        Y = np.array(Y)
        print(X.shape)
        print(Y.shape)
        
        Y = keras.utils.to_categorical(Y, num_classes = 4)

        
        history = self.model.fit(x=X, y=Y, epochs = 30, validation_split = 0.2)
        return self

    def learned_act(self, s):
        return np.ndarray.argmax(self.model.predict(np.expand_dims(s, axis=0)), axis=1)

In [165]:
imitating_a = ImitatingAgent(lr = 0.01, memory = new_memory)
imitating_a.imitate()


# Evaluation
test(imitating_a,env,epochs_test,prefix='cnn_test_explore')
#HTML(display_videos('cnn_test_explore10.mp4'))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_139 (Conv2D)          (None, 3, 3, 16)          448       
_________________________________________________________________
batch_normalization_3 (Batch (None, 3, 3, 16)          64        
_________________________________________________________________
conv2d_140 (Conv2D)          (None, 1, 1, 16)          2320      
_________________________________________________________________
flatten_69 (Flatten)         (None, 16)                0         
_________________________________________________________________
dense_95 (Dense)             (None, 4)                 68        
Total params: 2,900
Trainable params: 2,868
Non-trainable params: 32
_________________________________________________________________
None
(6030, 5, 5, 3)
(6030, 1)
Train on 4824 samples, validate on 1206 samples
Epoch 1/30
4824/4824 [==============================] - 3s 6

We can observe that the imitating agent overfits to the training set (much lower accuracy on the validation than on the training set). It can also be seen that the imitator's performance is worse than that of the imitated algorithm. This could be explained by the fact that the imitator isn't able to perfectly mimic the 'expert' (as can be seen from the accuracy being much lower than 1., particularly on the validation set).